In [1]:
# Setup Model
from torch import nn

class DirectionEmbeddingNet(nn.Module):
    def __init__(self, in_channels, direction_dim, embedding_dim, hidden_dim=64):
        """
        Args:
            in_channels: Number of input channels (e.g., number of semantic classes).
            direction_dim: Number of direction classes (e.g., 36).
            embedding_dim: Dimension of the embedding output.
            hidden_dim: Number of hidden channels in intermediate layers.
        """
        super(DirectionEmbeddingNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.direction_head = nn.Conv2d(hidden_dim, direction_dim, kernel_size=1)
        self.embedding_head = nn.Conv2d(hidden_dim, embedding_dim, kernel_size=1)

    def forward(self, x):
        """
        Args:
            x: Input semantic map of shape (B, in_channels, H, W).
        Returns:
            direction_map: Direction map of shape (B, direction_dim, H, W).
            embedding_map: Embedding map of shape (B, embedding_dim, H, W).
        """
        features = self.encoder(x)
        direction_map = self.direction_head(features)
        embedding_map = self.embedding_head(features)
        return direction_map, embedding_map

In [ ]:
# Setup Training
import os, sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from time import time
from tqdm import tqdm
from data.dataset import semantic_dataset  # Assuming this provides the dataset
from evaluation.angle_diff import calc_angle_diff  # For evaluation
import logging
from tensorboardX import SummaryWriter

from loss import DiscriminativeLoss

# Define the training function
def train_direction_embedding_model(args):
    if not os.path.exists(args['logdir']):
        os.makedirs(args['logdir'])
    logging.basicConfig(filename=os.path.join(args['logdir'], "results.log"),
                        filemode='w',
                        format='%(asctime)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S',
                        level=logging.INFO)
    logging.getLogger('shapely.geos').setLevel(logging.CRITICAL)

    logger = logging.getLogger()
    logger.addHandler(logging.StreamHandler(sys.stdout))
    # Create the dataset and dataloaders
    data_conf = {
        'num_channels': args['num_classes'] + 1,
        'image_size': args['image_size'],
        'xbound': args['xbound'],
        'ybound': args['ybound'],
        'zbound': args['zbound'],
        'dbound': args['dbound'],
        'thickness': args['thickness'],
        'angle_class': args['angle_class'],
    }
    train_loader, val_loader = semantic_dataset(args['version'], args['dataroot'], data_conf, args['bsz'], args['nworkers'])

    # Initialize the model
    model = DirectionEmbeddingNet(
        in_channels=args['num_classes'] + 1,
        direction_dim=args['angle_class'] + 1,
        embedding_dim=args['embedding_dim']
    ).cuda()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = nn.DataParallel(model).to(device)

    # Define the loss functions and optimizer
    direction_criterion = nn.BCELoss(reduction='none')
    embedding_criterion = DiscriminativeLoss(args['embedding_dim'], args['delta_v'], args['delta_d']).cuda()
    optimizer = optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    scheduler = StepLR(optimizer, 10, 0.1)

    counter = 0
    counter2 = 0
    writer = SummaryWriter(log_dir=args['logdir'])    

    # Store best model in a variable
    best_model = None
    best_val_loss = float('inf')

    # Training loop
    for epoch in range(args['nepochs']):
        model.train()
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch [{epoch+1}/{args['nepochs']}]")
        for _, (_, _, _, _, _, _, _, _, _, _, semantic_gt, embedding_gt, direction_gt) in progress_bar:
            t0 = time()
            # Move data to GPU
            semantic_gt = semantic_gt.cuda().float()  # Input
            direction_gt = direction_gt.cuda()  # Direction labels
            embedding_gt = embedding_gt.cuda()  # Embedding labels

            # Forward pass
            optimizer.zero_grad()
            direction_pred, embedding_pred = model(semantic_gt)

            # Compute losses
            # direction_loss = direction_criterion(direction_pred, direction_gt.argmax(dim=1))  # CrossEntropy expects class indices
            lane_mask = (1 - direction_gt[:, 0]).unsqueeze(1)
            direction_loss = direction_criterion(torch.softmax(direction_pred, 1), direction_gt)
            direction_loss = (direction_loss * lane_mask).sum() / (lane_mask.sum() * direction_loss.shape[1] + 1e-6)
            angle_diff = calc_angle_diff(direction_pred, direction_gt, args['angle_class'])
            
            var_loss, dist_loss, reg_loss = embedding_criterion(embedding_pred, embedding_gt)
            # loss = direction_loss + embedding_loss  # Combine losses
            loss = var_loss * args['scale_var'] + dist_loss * args['scale_dist'] + direction_loss * args['scale_direction']

            # Backward pass and optimization
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])
            optimizer.step()
            counter += 1
            t1 = time()
            if counter % 10 == 0:
                write_to_tensorboard('train', counter, writer, t0, direction_loss, angle_diff, var_loss, dist_loss, reg_loss, loss, t1)

            progress_bar.set_postfix({'loss': loss.item(), 'direction_loss': direction_loss.item(), 'final_loss': loss.item()})

        # Validation loop
        model.eval()
        with torch.no_grad():
            total_loss = 0
            total_direction_loss = 0
            total_angle_diff = 0
            val_progress_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc="Validation")
            for _, (_, _, _, _, _, _, _, _, _, _, semantic_gt, embedding_gt, direction_gt) in val_progress_bar:
                t0 = time()
                semantic_gt = semantic_gt.cuda().float()
                direction_gt = direction_gt.cuda()
                embedding_gt = embedding_gt.cuda()

                direction_pred, embedding_pred = model(semantic_gt)

                lane_mask = (1 - direction_gt[:, 0]).unsqueeze(1)
                direction_loss = direction_criterion(torch.softmax(direction_pred, 1), direction_gt)
                direction_loss = (direction_loss * lane_mask).sum() / (lane_mask.sum() * direction_loss.shape[1] + 1e-6)
                angle_diff = calc_angle_diff(direction_pred, direction_gt, args['angle_class'])
                
                var_loss, dist_loss, reg_loss = embedding_criterion(embedding_pred, embedding_gt)
                loss = var_loss * args['scale_var'] + dist_loss * args['scale_dist'] + direction_loss * args['scale_direction']

                total_loss += loss.item()
                total_direction_loss += direction_loss.item()
                total_angle_diff += angle_diff
                val_progress_bar.set_postfix({'val_loss': loss.item(), 'direction_loss': direction_loss.item()})
                
                counter2 += 1
                t1 = time()
                if counter % 10 == 0:
                    write_to_tensorboard('val', counter, writer, t0, direction_loss, angle_diff, var_loss, dist_loss, reg_loss, loss, t1)
            avg_loss = total_loss / len(val_loader)
            if avg_loss < best_val_loss:
                best_model = model
                print(f"Best model saved with validation loss: {avg_loss:.4f}")
                best_val_loss = avg_loss
            avg_direction_loss = total_direction_loss / len(val_loader)
            # avg_embedding_loss = total_embedding_loss / len(val_loader)
            avg_angle_diff = total_angle_diff / len(val_loader)
            print(f"Validation Loss: {avg_loss:.4f}, Direction Loss: {avg_direction_loss:.4f}, Average Angle Difference: {avg_angle_diff:.4f}")

        # Save the model checkpoint
        torch.save(model.state_dict(), os.path.join(args['logdir'], f"direction_embedding_model_epoch_{epoch+1}.pth"))
        print(f"Model saved for epoch {epoch+1}")
        scheduler.step()
    return best_model

def write_to_tensorboard(title, counter, writer, t0, direction_loss, angle_diff, var_loss, dist_loss, reg_loss, loss, t1):
    writer.add_scalar(f'{title}/step_time', t1 - t0, counter)
    writer.add_scalar(f'{title}/var_loss', var_loss, counter)
    writer.add_scalar(f'{title}/dist_loss', dist_loss, counter)
    writer.add_scalar(f'{title}/reg_loss', reg_loss, counter)
    writer.add_scalar(f'{title}/direction_loss', direction_loss, counter)
    writer.add_scalar(f'{title}/final_loss', loss, counter)
    writer.add_scalar(f'{title}/angle_diff', angle_diff, counter)

In [13]:
# Main function
if __name__ == "__main__":
    args = {
        "logdir": "./results/direction_embedding_net",
        "dataroot": "/home/yoganandam/data/nuScenes/v1.0-mini/",
        "version": "v1.0-mini",  # Choices: ["v1.0-trainval", "v1.0-mini"]
        "nepochs": 30,
        "bsz": 4,
        "nworkers": 10,
        "lr": 1e-3,
        "weight_decay": 1e-7,
        "num_classes": 3,  # Number of semantic classes
        "angle_class": 36,
        "embedding_dim": 16,
        "image_size": [128, 352],
        "xbound": [-30.0, 30.0, 0.15],
        "ybound": [-15.0, 15.0, 0.15],
        "zbound": [-10.0, 10.0, 20.0],
        "dbound": [4.0, 45.0, 1.0],
        "thickness": 5,
        "delta_v": 0.5,
        "delta_d": 3.0,
        "scale_var": 1.0,
        "scale_dist": 1.0,
        "scale_direction": 1.0,
        "max_grad_norm": 5.0,
    }
    print(args)
    best_model = train_direction_embedding_model(args)

{'logdir': './results/direction_embedding_net', 'dataroot': '/home/yoganandam/data/nuScenes/v1.0-mini/', 'version': 'v1.0-mini', 'nepochs': 30, 'bsz': 4, 'nworkers': 10, 'lr': 0.001, 'weight_decay': 1e-07, 'num_classes': 3, 'angle_class': 36, 'embedding_dim': 16, 'image_size': [128, 352], 'xbound': [-30.0, 30.0, 0.15], 'ybound': [-15.0, 15.0, 0.15], 'zbound': [-10.0, 10.0, 20.0], 'dbound': [4.0, 45.0, 1.0], 'thickness': 5, 'delta_v': 0.5, 'delta_d': 3.0, 'scale_var': 1.0, 'scale_dist': 1.0, 'scale_direction': 1.0, 'max_grad_norm': 5.0}


Validation: 100%|██████████| 21/21 [00:07<00:00,  2.67it/s, val_loss=2.01, direction_loss=0.071]  

Best model saved with validation loss: 2.2910
Validation Loss: 2.2910, Direction Loss: 0.1048, Average Angle Difference: 26.6373
Model saved for epoch 1



Validation: 100%|██████████| 21/21 [00:08<00:00,  2.54it/s, val_loss=1.98, direction_loss=0.0676]

Best model saved with validation loss: 2.2236
Validation Loss: 2.2236, Direction Loss: 0.1038, Average Angle Difference: 26.5776
Model saved for epoch 2


...

Validation Loss: 2.1990, Direction Loss: 0.0974, Average Angle Difference: 21.2494
Model saved for epoch 28



Validation: 100%|██████████| 21/21 [00:07<00:00,  2.72it/s, val_loss=1.94, direction_loss=0.0633]

Validation Loss: 2.1991, Direction Loss: 0.0974, Average Angle Difference: 21.2689
Model saved for epoch 29



Validation: 100%|██████████| 21/21 [00:08<00:00,  2.61it/s, val_loss=1.94, direction_loss=0.0634] 

Validation Loss: 2.1990, Direction Loss: 0.0974, Average Angle Difference: 21.2194
Model saved for epoch 30


In [ ]:
# Export results
import argparse
import mmcv
import tqdm
import torch

from data.dataset import semantic_dataset
from data.const import NUM_CLASSES
from model import get_model
from postprocess.vectorize import vectorize
import matplotlib.pyplot as plt

def gen_dx_bx(xbound, ybound):
    dx = [row[2] for row in [xbound, ybound]]
    bx = [row[0] + row[2] / 2.0 for row in [xbound, ybound]]
    nx = [(row[1] - row[0]) / row[2] for row in [xbound, ybound]]
    return dx, bx, nx

def export_to_json(model, val_loader, angle_class, args):
    submission = {
        "meta": {
            "use_camera": True,
            "use_lidar": False,
            "use_radar": False,
            "use_external": False,
            "vector": True,
        },
        "results": {}
    }

    dx, bx, nx = gen_dx_bx(args['xbound'], args['ybound'])

    model.eval()
    with torch.no_grad():
        val_progress_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc="Validation")
        for batchi, (_, _, _, _, _, _, _, _, _, _, segmentation, embedding_gt, direction_gt) in val_progress_bar:
            segmentation = segmentation.cuda().float()
            direction, embedding = model(segmentation)
            for si in range(segmentation.shape[0]):
                coords, confidences, line_types = vectorize(segmentation[si], embedding[si], direction[si], angle_class)
                # vectors = []
                # for coord, confidence, line_type in zip(coords, confidences, line_types):
                #     vector = {'pts': coord * dx + bx, 'pts_num': len(coord), "type": line_type, "confidence_level": confidence}
                #     vectors.append(vector)
                # rec = val_loader.dataset.samples[batchi * val_loader.batch_size + si]
                # submission['results'][rec['token']] = vectors
                for coord in coords:
                    plt.plot(coord[:, 0], coord[:, 1], linewidth=5)

                plt.xlim((0, segmentation.shape[3]))
                plt.ylim((0, segmentation.shape[2]))
                # plt.imshow(car_img, extent=[segmentation.shape[3]//2-15, segmentation.shape[3]//2+15, segmentation.shape[2]//2-12, segmentation.shape[2]//2+12])

                img_name = f'/home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval{batchi:06}_{si:03}.jpg'
                # print('saving', img_name)
                plt.savefig(img_name)
                plt.close()

    mmcv.dump(submission, args['output'])
    

/home/yoganandam/miniconda3/envs/hdmapnet/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/home/yoganandam/miniconda3/envs/hdmapnet/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/yoganandam/miniconda3/envs/hdmapnet/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# Main function
if __name__ == "__main__":
    args = {
        "logdir": "./results/direction_embedding_net",
        "dataroot": "/home/yoganandam/data/nuScenes/v1.0-mini/",
        "version": "v1.0-mini",  # Choices: ["v1.0-trainval", "v1.0-mini"]
        "nepochs": 30,
        "bsz": 4,
        "nworkers": 10,
        "lr": 1e-3,
        "weight_decay": 1e-7,
        "num_classes": 3,  # Number of semantic classes
        "angle_class": 36,
        "embedding_dim": 16,
        "image_size": [128, 352],
        "xbound": [-30.0, 30.0, 0.15],
        "ybound": [-15.0, 15.0, 0.15],
        "zbound": [-10.0, 10.0, 20.0],
        "dbound": [4.0, 45.0, 1.0],
        "thickness": 5,
        "delta_v": 0.5,
        "delta_d": 3.0,
        "scale_var": 1.0,
        "scale_dist": 1.0,
        "scale_direction": 1.0,
        "max_grad_norm": 5.0,
    }
    print(args)
    data_conf = {
        'num_channels': args['num_classes'] + 1,
        'image_size': args['image_size'],
        'xbound': args['xbound'],
        'ybound': args['ybound'],
        'zbound': args['zbound'],
        'dbound': args['dbound'],
        'thickness': args['thickness'],
        'angle_class': args['angle_class'],
    }
    train_loader, val_loader = semantic_dataset(args['version'], args['dataroot'], data_conf, args['bsz'], args['nworkers'])

{'logdir': './results/direction_embedding_net', 'dataroot': '/home/yoganandam/data/nuScenes/v1.0-mini/', 'version': 'v1.0-mini', 'nepochs': 30, 'bsz': 4, 'nworkers': 10, 'lr': 0.001, 'weight_decay': 1e-07, 'num_classes': 3, 'angle_class': 36, 'embedding_dim': 16, 'image_size': [128, 352], 'xbound': [-30.0, 30.0, 0.15], 'ybound': [-15.0, 15.0, 0.15], 'zbound': [-10.0, 10.0, 20.0], 'dbound': [4.0, 45.0, 1.0], 'thickness': 5, 'delta_v': 0.5, 'delta_d': 3.0, 'scale_var': 1.0, 'scale_dist': 1.0, 'scale_direction': 1.0, 'max_grad_norm': 5.0}


In [4]:
best_model = DirectionEmbeddingNet(
        in_channels=args['num_classes'] + 1,
        direction_dim=args['angle_class'] + 1,
        embedding_dim=args['embedding_dim']
    ).cuda()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model = nn.DataParallel(best_model).to(device)
best_model.load_state_dict(torch.load('/home/yoganandam/workspace/Online-HDMap/HDMapNet/results/direction_embedding_net/direction_embedding_model_epoch_30.pth'))

<All keys matched successfully>

In [5]:
# Export results
import argparse
import mmcv
from tqdm import tqdm
import torch

from data.dataset import semantic_dataset
from data.const import NUM_CLASSES
from model import get_model
from postprocess.vectorize import vectorize
args = {
    "dataroot": "dataset/nuScenes/",
    "version": "v1.0-mini",
    "model": "HDMapNet_cam",
    "bsz": 4,
    "nworkers": 10,
    "modelf": None,
    "thickness": 5,
    "image_size": [128, 352],
    "xbound": [-30.0, 30.0, 0.15],
    "ybound": [-15.0, 15.0, 0.15],
    "zbound": [-10.0, 10.0, 20.0],
    "dbound": [4.0, 45.0, 1.0],
    "embedding_dim": 16,
    "angle_class": 36,
    "output": "output.json"
}
data_conf = {
    'num_channels': NUM_CLASSES + 1,
    'image_size': args['image_size'],
    'xbound': args['xbound'],
    'ybound': args['ybound'],
    'zbound': args['zbound'],
    'dbound': args['dbound'],
    'thickness': args['thickness'],
    'angle_class': args['angle_class'],
}
export_to_json(best_model, val_loader, args['angle_class'], args)

Validation:   0%|          | 0/21 [00:00<?, ?it/s]

saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000000_000.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000000_001.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000000_002.jpg


Validation:   5%|▍         | 1/21 [00:15<05:13, 15.70s/it]

saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000000_003.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000001_000.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000001_001.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000001_002.jpg


Validation:  10%|▉         | 2/21 [00:24<03:45, 11.86s/it]

...

saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000018_003.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000019_000.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000019_001.jpg
saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000019_002.jpg


Validation:  95%|█████████▌| 20/21 [02:14<00:05,  5.91s/it]

saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000019_003.jpg


Validation: 100%|██████████| 21/21 [02:17<00:00,  6.53s/it]

saving /home/yoganandam/workspace/Online-HDMap/HDMapNet/results/imgs/eval000020_000.jpg
